# Healdette Documentation Update Notebook

This notebook automates the process of updating project documentation, including version numbers, changelog, README, security policy, and API documentation.

## 1. Update Version Numbers

First, we'll update version numbers across the project files. This section will help maintain consistent versioning throughout the codebase.

In [ ]:
import re
from pathlib import Path

def update_version(new_version):
    # Find all Python files that might contain version info
    files_to_check = list(Path('.').rglob('*.py'))
    
    version_pattern = r'__version__\s*=\s*[\'"]([^\'"]*)[\'"]'
    
    for file_path in files_to_check:
        content = file_path.read_text()
        if '__version__' in content:
            updated_content = re.sub(version_pattern, 
                                   f'__version__ = "{new_version}"', 
                                   content)
            file_path.write_text(updated_content)
            print(f"Updated version in {file_path}")

# Example usage:
# update_version('1.0.1')

## 2. Update Changelog

Now let's update the CHANGELOG.md file by parsing git commits and organizing them according to the Keep a Changelog format.

In [ ]:
import subprocess
from datetime import datetime

def get_git_commits(since_tag=None):
    cmd = ['git', 'log', '--pretty=format:%h|%s|%ad', '--date=short']
    if since_tag:
        cmd.append(f'{since_tag}..HEAD')
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    commits = result.stdout.strip().split('\n')
    
    categorized_commits = {
        'Added': [],
        'Changed': [],
        'Deprecated': [],
        'Removed': [],
        'Fixed': [],
        'Security': []
    }
    
    for commit in commits:
        hash_, msg, date = commit.split('|')
        category = 'Changed'  # default category
        
        # Categorize based on commit message
        lower_msg = msg.lower()
        if 'add' in lower_msg or 'new' in lower_msg:
            category = 'Added'
        elif 'deprecat' in lower_msg:
            category = 'Deprecated'
        elif 'remove' in lower_msg or 'delete' in lower_msg:
            category = 'Removed'
        elif 'fix' in lower_msg or 'bug' in lower_msg:
            category = 'Fixed'
        elif 'secur' in lower_msg:
            category = 'Security'
        
        categorized_commits[category].append(f"- {msg} ({hash_})")
    
    return categorized_commits, date

def update_changelog(version):
    commits, date = get_git_commits()
    
    changelog_content = f"""## [{version}] - {date}
    
"""
    
    for category, entries in commits.items():
        if entries:
            changelog_content += f"### {category}\n"
            changelog_content += "\n".join(entries) + "\n\n"
    
    # Read existing CHANGELOG.md
    try:
        with open('CHANGELOG.md', 'r') as f:
            existing_content = f.read()
    except FileNotFoundError:
        existing_content = """# Changelog
All notable changes to this project will be documented in this file.

The format is based on [Keep a Changelog](https://keepachangelog.com/en/1.0.0/).
"""
    
    # Insert new changes after the header
    header_end = existing_content.find('\n\n')
    updated_content = (existing_content[:header_end + 2] + 
                      changelog_content + 
                      existing_content[header_end + 2:])
    
    with open('CHANGELOG.md', 'w') as f:
        f.write(updated_content)
    
    print(f"Updated CHANGELOG.md with version {version}")

# Example usage:
# update_changelog('1.0.1')

## 3. Update README Documentation

Let's update the README.md with new features, requirements, and usage examples.

In [ ]:
import pkg_resources
import inspect
import importlib

def update_readme():
    # Get installed dependencies
    requirements = [str(r) for r in pkg_resources.working_set]
    
    # Import main module to get documentation
    module = importlib.import_module('main')
    
    # Generate new README content
    readme_content = f"""# Healdette

A powerful tool for antibody pipeline analysis and optimization.

## Features

{inspect.getdoc(module)}

## Installation

```bash
pip install -r requirements.txt
```

## Dependencies

{chr(10).join(['- ' + r for r in requirements])}

## Usage

Basic example:

```python
from healdette import analyze_antibody

# Initialize analysis
result = analyze_antibody('sample.fasta')

# Run optimization
optimized = result.optimize()

# Export results
optimized.save('output.json')
```

## Documentation

For full documentation, visit our [documentation site](docs/).

## Contributing

Please read [CONTRIBUTING.md](CONTRIBUTING.md) for details on our code of conduct and the process for submitting pull requests.

## Security

See [SECURITY.md](SECURITY.md) for our security policy and reporting procedures.

## License

This project is licensed under the MIT License - see the [LICENSE](LICENSE) file for details.
"""

    with open('README.md', 'w') as f:
        f.write(readme_content)
    
    print("Updated README.md")

# Example usage:
# update_readme()

## 4. Security Policy Updates

Update the security policy with vulnerability reporting procedures and security considerations.

In [ ]:
def update_security_policy():
    security_content = """# Security Policy

## Supported Versions

| Version | Supported          |
| ------- | ------------------ |
| 1.0.x   | :white_check_mark: |
| < 1.0   | :x:                |

## Reporting a Vulnerability

We take the security of Healdette seriously. If you believe you have found a security vulnerability, please report it to us as described below.

### Reporting Process

1. **DO NOT** create a public GitHub issue for the vulnerability.
2. Email your findings to security@healdette.org
3. Include detailed steps to reproduce the issue
4. We will respond within 48 hours with next steps

### What to Include in Your Report

- Type of issue (buffer overflow, SQL injection, cross-site scripting, etc.)
- Full paths of source files related to the manifestation of the issue
- Location of the affected source code (tag/branch/commit or direct URL)
- Step-by-step instructions to reproduce the issue
- Proof-of-concept or exploit code (if possible)
- Impact of the issue, including how an attacker might exploit it

## Security Features

- All data processing is done locally
- Secure storage of credentials
- Input validation and sanitization
- Regular security updates and patches

## Best Practices

1. Always keep Healdette updated to the latest version
2. Use strong authentication credentials
3. Validate all input data
4. Monitor system logs for unusual activity
5. Follow the principle of least privilege

## Security Updates

Security updates will be clearly marked in our version releases and documented in our changelog.
"""

    with open('SECURITY.md', 'w') as f:
        f.write(security_content)
    
    print("Updated SECURITY.md")

# Example usage:
# update_security_policy()

## 5. Generate API Documentation

Finally, let's generate updated API documentation from docstrings using Sphinx.

In [ ]:
import pdoc
import os

def generate_api_docs():
    # Create docs directory if it doesn't exist
    os.makedirs('docs/api', exist_ok=True)
    
    # Generate documentation
    modules = [
        'main',
        'generate_binders',
        'validate_ethics',
        'run_simulations',
        'extract_signature',
        'fuse_perspectives'
    ]
    
    for module_name in modules:
        module = importlib.import_module(module_name)
        
        # Generate HTML documentation
        html = pdoc.html(module_name)
        
        # Save to file
        with open(f'docs/api/{module_name}.html', 'w') as f:
            f.write(html)
        
        print(f"Generated API documentation for {module_name}")

    # Create index.html
    index_content = """<!DOCTYPE html>
<html>
<head>
    <title>Healdette API Documentation</title>
</head>
<body>
    <h1>Healdette API Documentation</h1>
    <ul>
"""
    
    for module_name in modules:
        index_content += f'        <li><a href="{module_name}.html">{module_name}</a></li>\n'
    
    index_content += """    </ul>
</body>
</html>
"""
    
    with open('docs/api/index.html', 'w') as f:
        f.write(index_content)
    
    print("Generated API documentation index")

# Example usage:
# generate_api_docs()

## Running the Documentation Update

To update all documentation, run all cells in sequence. Each section can also be run independently if you only need to update specific documentation components.

Remember to commit your changes after running the updates!